In [234]:
using Plots

In [235]:
# define a function that returns a Plots.Shape
rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])

rectangle (generic function with 1 method)

In [237]:
function plot_tdd_pattern(period_ms, dl_slots, ul_slots)
    total_duration_ms = 100  # Adjust the total duration as needed
    symbols_per_slot = 10  # Adjust the number of symbols per slot as needed

    dl_color = "blue"
    ul_color = "green"

    dl_slots_timeline = repeat([1; zeros(symbols_per_slot - 1)], dl_slots)
    ul_slots_timeline = repeat([2; zeros(symbols_per_slot - 1)], ul_slots)

    timeline = repeat(1:symbols_per_slot:total_duration_ms, inner=dl_slots + ul_slots); @show timeline
    pattern = vcat(dl_slots_timeline, ul_slots_timeline)

    # Create separate arrays for dl_colors and ul_colors
    dl_colors = [dl_color for _ in 1:dl_slots]; @show dl_colors;
    ul_colors = [ul_color for _ in 1:ul_slots]; @show ul_colors;

    # Concatenate dl_colors and ul_colors
    colors = vcat(dl_colors, "white", ul_colors); @show colors

    plot(timeline, pattern
        , linecolor=timeline
        , xlabel="Time (ms)"
        , ylabel="Slot Type"
        , label=["DL" "UL"])
    return timeline, colors
end

plot_tdd_pattern (generic function with 1 method)

In [238]:
# Configuration parameters
tdd_period_ms = 5
dl_slots = 2
ul_slots = 7

# Plot TDD pattern
# dl_ul_plot = 
timeline, colors = plot_tdd_pattern(tdd_period_ms, dl_slots, ul_slots)

# Save the figure to an image file
# savefig(dl_ul_plot, "dl_ul_plot.png")

timeline = [1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 11, 11, 11, 11, 11, 11, 11, 11, 21, 21, 21, 21, 21, 21, 21, 21, 21, 31, 31, 31, 31, 31, 31, 31, 31, 31, 41, 41, 41, 41, 41, 41, 41, 41, 41, 51, 51, 51, 51, 51, 51, 51, 51, 51, 61, 61, 61, 61, 61, 61, 61, 61, 61, 71, 71, 71, 71, 71, 71, 71, 71, 71, 81, 81, 81, 81, 81, 81, 81, 81, 81, 91, 91, 91, 91, 91, 91, 91, 91, 91]
dl_colors = ["blue", "blue"]
ul_colors = ["green", "green", "green", "green", "green", "green", "green"]
colors = ["blue", "blue", "white", "green", "green", "green", "green", "green", "green", "green"]


([1, 1, 1, 1, 1, 1, 1, 1, 1, 11  …  81, 91, 91, 91, 91, 91, 91, 91, 91, 91], ["blue", "blue", "white", "green", "green", "green", "green", "green", "green", "green"])

In [239]:
X = unique(timeline); @show X, size(X), typeof(X) 
Xₗ = colors

Wₛₜ = tdd_period_ms # 5 # ms
Hₛₜ = 1 # standard height

p = 
plot(0:1,0:1, legend=:false, yaxis=:false, opacity=.8, xlabel="Time (ms)")

for i in 1:length(X)
    dl = rectangle(Wₛₜ,Hₛₜ,X[i]//2,0); p = plot!(dl, color=Xₗ[i])
end

i = findall(x->x=="white", Xₗ); @show i
dl1 = rectangle(Wₛₜ//2,Hₛₜ,X[i]//2,0); p = plot!(dl1, color="white")
savefig(p,"images/slots.png");

(X, size(X), typeof(X)) = ([1, 11, 21, 31, 41, 51, 61, 71, 81, 91], (10,), Vector{Int64})
i = [3]


<img src=images/slots.png width='' height='' > </img>

In [240]:
function calculate_frame_duration(Δf, dl_slots, ul_slots)
    slot_duration = 1 / Δf
    slots_per_frame = dl_slots + ul_slots
    frame_duration_seconds = slot_duration * slots_per_frame
    frame_duration_milliseconds = frame_duration_seconds * 1e3  # Convert to milliseconds
    return frame_duration_milliseconds
end

calculate_frame_duration (generic function with 1 method)

In [241]:
# Example with 30 kHz subcarrier spacing
Δf = 30e3
dl_slots = 7
ul_slots = 2

frame_duration_ms = calculate_frame_duration(Δf, dl_slots, ul_slots)
println("Frame Duration: $frame_duration_ms milliseconds")

Frame Duration: 0.30000000000000004 milliseconds


In [242]:
# Cell 1 parameters
band1 = "n78"
arfcn1 = 641280
bandwidth1 = 40
subcarrier_spacing1 = 30

# Cell 2 parameters
band2 = "n48"
arfcn2 = 643000
bandwidth2 = 40
subcarrier_spacing2 = 30

# Frequency axis
frequencies = [arfcn1, arfcn2]
bandwidths = [bandwidth1, bandwidth2]

# Plotting cells on the frequency axis
p = plot(frequencies
    , zeros(length(frequencies))
    , seriestype=:scatter
    , label=""
    , xlabel="Frequency (MHz)"
    , ylabel="Cell"
    , title="Cell Frequencies"
    , size=(800, 600)
)

# Add cell information to the plot
for i in 1:length(frequencies)
    p = annotate!([(frequencies[i]-(i == 2 ? 500 : 0), 0.5, text("Cell $i\nBand: $(getproperty(Main, Symbol("band$i")))\nBandwidth: $(getproperty(Main, Symbol("bandwidth$i"))) MHz", :left))])
    if i < length(frequencies)
        p = plot!([(frequencies[i] + frequencies[i+1]) / 2, (frequencies[i] + frequencies[i+1]) / 2], [-0.5, 0.5], label="", linestyle=:dash, color=:black)
    end
end

# Show the plot
p = plot!()
# display(p)
savefig(p,"images/two_cell_freq_axis.png");

<img src=images/two_cell_freq_axis.png width='' height='' > </img>

In [232]:
using Latexify

In [243]:
function plot_5g_time_frame(subframes, total_duration_ms, subframe_duration_ms)

    subframe_start_times = 0:subframe_duration_ms:total_duration_ms
    subframe_end_times = subframe_start_times .+ subframe_duration_ms

    p₁ = plot(subframe_start_times, zeros(length(subframe_start_times)), seriestype=:scatter, label="", xlabel="Time (ms)", ylabel="Subframe", title="5G Time Frame", size=(600, 200))

    for i in 1:length(subframe_start_times)
        p₁= annotate!([(subframe_start_times[i] + subframe_end_times[i] / 100, 0.40, latexify("S_F$i"))])
        if i < length(subframe_start_times)
            p₁ = plot!([(subframe_start_times[i] + subframe_end_times[i]) / 2, (subframe_start_times[i] + subframe_end_times[i]) / 2], [-0.5, 0.5], label="", linestyle=:dash, color=:black)
        end
    end

    # Show the plot
    p₁ = plot!()
    # display(p₁)
    savefig(p₁,"images/5g_time_frame.png");
end

plot_5g_time_frame (generic function with 2 methods)

In [244]:
subframes = 10
total_duration_ms = 10  # Total duration of the time frame
subframe_duration_ms = 1  # Duration of each subframe

# Example: Plot a 5G time frame with 10 subframes
plot_5g_time_frame(subframes, total_duration_ms, subframe_duration_ms)

"/Users/valiha/Developer/mobsys/lab1/images/5g_time_frame.png"

<img src=images/5g_time_frame.png width='' height='' > </img>

In [47]:
function calculate_numerology(Δf)
    T_s = 1 / Δf  # Symbol duration
    T_slot = 14 * T_s  # Slot duration (considering 14 symbols per slot)
    T_frame = 10 * T_slot  # Frame duration (considering 10 slots per frame)

    return T_s, T_slot, T_frame
end

# Example usage
Δf = 15e3  # Subcarrier spacing in Hertz (15 kHz)
T_s, T_slot, T_frame = calculate_numerology(Δf)

@show round(T_s; digits = 4)
@show round(T_slot; digits = 4)
@show round(T_frame; digits = 3)
println("Symbol Duration: $T_s seconds")
println("Slot Duration: $T_slot seconds")
println("Frame Duration: $T_frame seconds")


round(T_s; digits = 4) = 0.0001
round(T_slot; digits = 4) = 0.0009
round(T_frame; digits = 3) = 0.009
Symbol Duration: 6.666666666666667e-5 seconds
Slot Duration: 0.0009333333333333333 seconds
Frame Duration: 0.009333333333333332 seconds
